In [2]:
import pandas as pd

data = pd.read_csv('data/non_annualized_KY_lo_hi.csv')
data.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),...,Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%),Unnamed: 19,Class
0,1276,Kentucky,Lexington,3/31/2015,Buffalo,9/17/2015,260,0.21,170,34,...,41.1,21.523529,15.455882,27.502941,0.205735,1,0,95.224000,NaN,0
1,1279,Kentucky,Lexington,3/31/2015,Saranac AR,9/17/2015,260,0.24,170,34,...,41.1,21.523529,15.455882,27.502941,0.205735,1,0,86.432000,NaN,0
2,1030,Kentucky,Lexington,8/9/2012,Arc,9/16/2015,259,0.26,1133,34,...,41.1,21.526471,15.508824,27.458824,0.210147,0,0,91.852041,NaN,0
3,1220,Kentucky,Lexington,8/9/2012,Evermore,9/12/2018,255,0.32,2225,30,...,138.7,23.033333,18.906667,27.833333,0.319600,0,0,66.400000,NaN,0
4,1267,Kentucky,Lexington,3/31/2015,Contender,9/17/2015,260,0.32,170,34,...,41.1,21.523529,15.455882,27.502941,0.205735,1,0,95.776000,NaN,0


In [3]:
from sdv.tabular import TVAE

model = TVAE()
model.fit(data)

In [4]:
new_data = model.sample(5000)

In [5]:
new_data.to_csv('data/VAE_5ksynth_0715_KY_KNN.csv')
new_data.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),...,Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%),Unnamed: 19,Class
0,822,Kentucky,Lexington,8/9/2012,Caliber,5/13/2013,125,1.45,223,226,...,305.9,11.736670,6.338495,17.954482,0.331804,0,1,98.003404,NaN,1
1,1036,Kentucky,Lexington,8/9/2012,CW 065030,7/8/2017,189,0.87,1716,31,...,163.7,22.143725,17.155325,26.775342,0.289080,0,0,95.316108,NaN,1
2,1217,Kentucky,Lexington,8/9/2012,Saranac AR,8/13/2015,223,0.70,225,31,...,165.2,23.768608,18.372763,27.974346,0.317020,0,0,94.600645,NaN,0
3,992,Kentucky,Lexington,8/9/2012,GA-ALFG-1,9/12/2018,254,0.76,2139,33,...,109.8,22.843712,18.162489,27.420778,0.253491,0,0,74.997384,NaN,0
4,1283,Kentucky,Lexington,3/31/2015,GA-505,7/8/2017,191,1.24,194,31,...,245.2,23.051598,17.521980,28.205458,0.269609,0,0,91.115581,NaN,1


In [6]:
# get aggregate data
targetDataLoc = 'data/non_annualized_GA_lo_hi.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = new_data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [7]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [8]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [9]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [#(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsRegressor(), knnParamGrid, "knnTup.pkl")]#,
              #(SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPClassifier(), nnParamGrid, "nnTup.pkl")]#,
             #(LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 8 # 11

In [10]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

ValueError: Classification metrics can't handle a mix of binary and continuous targets